In [1]:
import pandas as pd
#df=pd.read_csv('complete_data.csv')
df=pd.read_csv('split_threads (1).csv')
stopped=pd.read_csv('clean_emails_Nicolo.csv')
from names_dataset import NameDataset, NameWrapper
names=NameDataset()
def detect_name(string, r = 5000, c = 3):
    '''
    Function that returns True if the provided string contains a name (for US present in top 3 countries)
    r: Minimum position in the ranking of popular names/last names in the US
    c: Minimum ranking of popularity of the word as a name/last name in the US
    '''
    words = string.split(" ")
    l = False
    for w in words:

        x = names.search(w)['first_name']
        if x is None:# word not found in names dictionary
            f = False
        elif x['rank']['United States'] is None:# Name no frequent in US
            f = False
        elif x['rank']['United States'] > r:# Name unfrequent in US 
            f = False
        else:
            y = x['country']
            countries_first_name = list({k: v for k, v in sorted(y.items(), key=lambda item: item[1], reverse = True)}.keys())[0:c]
            f = 'United States' in countries_first_name
        
        x = names.search(w)['last_name']
        if x is None:# word not found in names dictionary
            p = False
        elif x['rank']['United States'] is None:# Last name no frequent in US
            p = False
        elif x['rank']['United States'] > r:# Last name unfrequent in US 
            p = False
        else:
            y = x['country']
            countries_last_name = list({k: v for k, v in sorted(y.items(), key=lambda item: item[1], reverse = True)}.keys())[0:c]
            p = 'United States' in countries_last_name     
        
        l = l | ( f | p )
    return l


In [ ]:
salutations=[
'My Best',
'My best to you',
'Have a nice day',
'Hope your day is going well',
'Faithfully',
'All Best' ,
'All the best',
'Best Wishes',
'Bests',
'Best Regards',
'Regards',
'Rgds',
'Warm Regards',
'Warmest Regards',
'Warmest',
'Warmly',
'Take care',
'Thanks',
'Thanks so much',
'Thanks!',
'Thank you',
'Thank you!',
'Thanks a bunch',
'Many thanks',
'Thanks for your consideration',
'Thx',
'Hope this helps',
'Looking forward',
'Rushing',
'In haste',
'Be well',
'Peace',
'Yours Truly',
'Yours',
'Very Truly Yours',
'Sincerely',
'Sincerely Yours',
'Cheers!',
'faithfully'
'Ciao',
'Love',
'Lots of love',
'Hugs',
'High five from down low',
'Take it easy bro',
'See you around',
'Have a wonderful bountiful lustful day',
'Sent from my iPhone',
'Sent from iOS']
#list of common salutations
from tqdm import tqdm
from difflib import SequenceMatcher
#function that tells us how similar two strings are similar
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()
c=False
counter=0
rest=[]
words=[]
em=[]
se=[]
way=[]
prev=[]
indexes=[]
dete=[]
for index,row in tqdm(df.iterrows()):
    #split the body in line
    body=row['email'].split('\n')
    #check if the body is at least three (could be avoided in this case)
    if len(body)>3:
        #for line in the body 
        for j,line in enumerate(body):
            #if we are in the bottom half of the email we go forth. Assumption: salutation are in the bottom half
            if j>(1.5*len(body))/3:
                #if the line is less than 4 items, (probably a salutation) and the previous line is empty
                #and body[j-1]=='':
                if len(line.split())<=4:
                    s = ''.join(filter(str.isalnum, line))
                    for item in salutations:
                        #we wliminate anything that is punctuation or not letters,/numbers
                        #we get the similarity for each item in the salutation list. If we get good score we go on
                        if similar(s.lower(),item.lower())>0.65 and row['email_id'] not in rest:
                            #code for checking any len(3) salutation
                            #let's count how many email have this salutatiion
                            counter+=1
                            em.append(row['email'])
                            rest.append(row['email_id'])
                            indexes.append(j)
                            se.append(s)
                            way.append('salutation')
                            dete.append(item)
                            #let's get the word that triggered our salutation to compare
                            words.append(s.lower()+'--------->'+item.lower())
                            #Early stopping system to avoid  (change index for early stopping)
                            if counter==100000000 or index>5111000:
                                c=True
                    for item_word in line.split():
                        if item_word[0].isupper() and item_word[1:].islower():
                            try:
                                if detect_name(item_word) and row['email_id'] not in rest:
                                    counter+=1
                                    em.append(row['email'])
                                    rest.append(row['email_id'])
                                    indexes.append(j)
                                    se.append(s)
                                    dete.append(item_word)
                                    way.append('name search')
                            except:
                                #print('error')
                                continue
        if c==True:
            break 